In [ ]:
import os
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit

In [ ]:
# Only needed for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [ ]:
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
math_server_file_path = Path(notebook_dir).parent / "test/interop/mcp/math_server.py"
math_server_file_path

In [ ]:
server_params = StdioServerParameters(
    command="python",
    args=[str(math_server_file_path)],
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()

    toolkit = await create_toolkit(session=session)
    agent = AssistantAgent(name="assistant", llm_config={"model": "gpt-4o-mini", "api_type": "openai"})
    toolkit.register_for_llm(agent)

    await agent.a_run(
        message="Add 123223 and 456789",
        tools=toolkit.tools,
        max_turns=2,
        user_input=False,
    )